# 순차 데이터 (sequential data)

순차 데이터: 텍스트나 시계열 데이터(time series data)와 같이 순서에 의미가 있는 데이터
- 시계열 데이터: 일정한 시간 간격으로 기록된 데이터

지금까지 우리가 다뤘던 데이터는 입력 순서와는 상관이 없었다.
- FC 신경망이나 CNN은 하나의 샘플을 사용하여 정방향 계산을 수행하고 나면,
- 그 샘플은 버려지고 다음 샘플을 처리할 때 재사용하지 않는다.
- --> 피드포워드 신경망 (feedforward neural network)

그러나 이 장에서 다루고자 하는 텍스트 리뷰는 단어의 순서가 중요한 순차 데이터이다.
- 순차 데이터를 다루기 위해서는 이전에 입력한 데이터를 기억하는 기능이 필요하다.
- 다음 샘플을 위해서 이전 데이터가 신경망 층에 순환될 필요가 있다.
- --> 순환 신경망 (recurrent neural network; RNN)


# 순환 신경망

일반적인 FC 신경망과 거의 비슷하다.
- 뉴런의 출력이 다시 자기 자신으로 전달되는 고리 하나만 추가하면 됨

샘플 $x_1, x_2, x_3$에 대한 순환 뉴런 하나를 생각해보자.
- 먼저 $x_1$에 대한 출력 $h_1$을 만든다.
    - $h_1$은 $x_1$에 대한 정보를 어느 정도 갖고 있다.
- 그 출력 $h_1$은 $x_2$에 대한 출력 $h_2$를 만들 때에도 이용된다.
    - $h_2$은 $x_2, x_1$에 대한 정보를 어느 정도 갖고 있다.
- 그 출력 $h_2$은 $x_3$에 대한 출력 $h_3$를 만들 때에도 이용된다.
    - $h_3$은 $x_3, x_2, x_1$에 대한 정보를 어느 정도 갖고 있다.

위와 같이 샘플을 처리하는 단계 하나하나를 타임스텝(timestep)이라고 한다.
- 순환 신경망은 이전 타입스텝의 샘플에 대한 정보를 어느 정도 기억하지만,
- 타임스텝이 오래될수록 순환되는 정보는 희미해진다.

순환 신경망에서는 층(layer)을 특별히 셀(cell)이라고 부른다.
- 한 셀에는 여러 뉴런이 있지만, 순환 관계도를 그리기 복잡하기 때문에,
- 뉴런을 다 표시하지 않고 하나의 셀로 층을 표현한다.

또한, 셀의 출력을 은닉 상태(hidden state)라고 부른다.

또한, RNN에서도 활성화 함수가 반드시 필요하며, 일반적으로 tanh 함수를 사용한다.
- 시그모이드와 비슷한 모양이지만 치역의 범위가 (-1, 1)


---


RNN은 순환하는 구조가 추가되었기 때문에, 그 구조에 대한 가중치가 추가된다.
- $h_n = (샘플 입력x_n)×w_x + (이전 타임스텝의 은닉 상태 h_{n-1})×w_h + bias$
- 가중치 $w_h$는 타임스텝에 따라 변화하는 뉴런의 출력을 학습한다.
- 맨 처음 타임스텝1에서 사용하는 $h_0$은 단순히 0으로 초기화하여 이용한다.

# 셀의 가중치와 입출력

샘플 $x_1, x_2, x_3, x_4$를 입력으로 받는, 3개의 순환 뉴런 $r_1, r_2, r_3$으로 이루어진 셀 하나를 생각해보자.
- 가중치 $w_x$: 입력층과 셀은 모두 완전 연결이므로, 그 크기는 $3\times4=12$이다.
- 가중치 $w_h$: 순환층에서 은닉 상태는 자신을 포함한 모든 순환 뉴런으로 전파된다. 3개의 은닉 상태가 3개의 순환 뉴런으로 되돌아오므로, 그에 대한 가중치는 총 $3^2=9$개 필요하다.
- 절편 $bias$: 각 뉴런마다 하나의 절편이 포함되므로, 총 3개 존재한다.
- --> 위 셀의 모델 파라미터 수 $= 12 + 9 + 3 = 24$

다음으로는 순환층의 입출력에 대해 생각해보자.
- 순환층은 일반적으로 샘플마다 2개의 차원을 갖는다.
    - 보통, 하나의 샘플을 하나의 시퀀스(sequence)라고 부른다.
        - 시퀀스 안에는 여러 아이템이 들어있다.
        - 시퀀스의 길이는 타임스텝의 길이와 같다.
    - 예: 하나의 샘플(시퀀스) "I am a boy"
        - 이 샘플은 4개의 단어로 이루어진다.
        - 각 단어는 크기가 3인 차원 하나로 표현이 가능하다고 가정
        - --> 이 입력의 크기: (1, 4, 3)
    - (샘플 개수, 시퀀스 길이(타임스텝), 단어 표현)
- 어떤 입력이 순환층을 통과하면, 두, 세 번째 차원이 사라지고 순환층의 뉴런 개수만큼 출력된다.
    - 즉, 시퀀스 길이와 단어 표현의 2차원 배열이 1차원 배열로 축소된다.
    - 헷갈릴 수 있는 부분
        - 순환층은 매 타임스탭마다 출력을 만드는 것이 아니라,
        - 은닉 정보를 계속 순환시키다가 맨 마지막 은닉 상태(h_f)만을 출력하는 것

RNN도 FC 신경망이나 CNN처럼 여러 개의 층을 쌓을 수 있다.
- 셀의 입력은 (타임스텝, 단어 표현)으로 이루어진 2차원 배열이어야 한다. (배치차원 제외)
- 그러므로 마지막 셀을 제외한 모든 셀은 $h_f$만 출력해서는 안되고, 모든 타임스텝에 대한 은닉 상태를 출력해야 한다.
    - (샘플 수, 타임스텝, 단어 표현) + Rcrr1 --> (샘플 수, 타임스텝, Rcrr1의 뉴런 수)
        - 타임스텝 * Rcrr1의 뉴런 수 == 전체 은닉 상태 수
    - (샘플 수, 타임스텝, Rcrr1의 뉴런 수) + Rcrr2 --> (샘플 수, 뉴런 수)
        - 뉴런 수 = 마지막 타임스텝의 은닉 상태 수

출력층은 CNN과 마찬가지로 마지막엔 밀집층을 둔다.
- 다중 분류면 클래스 개수만큼 뉴런을 두고, softmax
- 이중 분류면 하나의 뉴런을 두고, sigmoid
- CNN과는 다르게 마지막 셀의 출력이 1차원이기 때문에, Flatten을 걸 필요가 없다.